CC5213 - Recuperación de Información Multimedia
==


Ejemplos Deteccion de Bordes
==

In [ ]:
import sys
import os
import numpy
import cv2
from utils import utils

print("Usando OpenCV {} con Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor, sys.version_info.micro))


Ejemplo 1 - OTSU
==


In [ ]:
def ejemplo(filename):
    imagen_color = utils.abrir_imagen(filename)
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    threshold, imagen_bin = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    window_name = os.path.basename(filename)
    utils.mostrar_imagen(window_name, imagen_color)
    utils.mostrar_imagen(window_name + " BIN", imagen_bin)
    print ("  threshold_otsu={}".format(threshold))

filenames = utils.ui_select_filenames()

if len(filenames) > 0:
    for filename in filenames:
        ejemplo(filename)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
print("FIN")

Ejemplo 2 - Sobel
==

In [ ]:
def ejemplo(filename):
    global sobel_threshold, delta
    capture = utils.abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        utils.mostrar_imagen("VIDEO", frame_gris)
        #calcular filtro de sobel
        sobelX = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
        sobelY = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
        utils.mostrar_imagen("X", utils.escalar(sobelX, valorAbsoluto=True, escalarMin0Max255=True))
        utils.mostrar_imagen("Y", utils.escalar(sobelY, valorAbsoluto=True, escalarMin0Max255=True))
        #magnitud del gradiente
        magnitud = numpy.sqrt(numpy.square(sobelX) + numpy.square(sobelY) )
        utils.mostrar_imagen("MAGNITUD GRADIENTE",  utils.escalar(magnitud, escalarMin0Max255=True))
        #aproximacion de la magnitud del gradiente
        aprox = numpy.abs(sobelX) + numpy.abs(sobelY)
        utils.mostrar_imagen("APROX GRADIENTE",  utils.escalar(aprox, escalarMin0Max255=True))
        #umbral sobre la magnitud del gradiente
        retval, bordes = cv2.threshold(magnitud, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
        utils.agregar_texto(bordes, "th={}".format(sobel_threshold))
        utils.mostrar_imagen("BORDES", bordes)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            sobel_threshold += delta
        elif key == ord('z'):
            if sobel_threshold - delta > 0:
                sobel_threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sobel_threshold = 51
delta = 5

filename = utils.ui_select_video()
ejemplo(filename)

print("FIN")

Ejemplo 3 - CANNY
==

In [ ]:
def ejemplo(filename):
    global canny_threshold_1, canny_threshold_2, delta
    capture = utils.abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        utils.mostrar_imagen("VIDEO", frame_gris)
        #calcular canny
        frame_canny = cv2.Canny(frame_gris, threshold1=canny_threshold_1, threshold2=canny_threshold_2)
        utils.agregar_texto(frame_canny, "th={}-{}".format(canny_threshold_1, canny_threshold_2))
        utils.mostrar_imagen("CANNY", frame_canny)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            canny_threshold_1 += delta
        elif key == ord('z'):
            if canny_threshold_1 - delta > 0:
                canny_threshold_1 -= delta
        elif key == ord('s'):
            canny_threshold_2 += delta
        elif key == ord('x'):
            if canny_threshold_2 - delta > 0:
                canny_threshold_2 -= delta
        else:
            print("unknown key '{}' ({})".format(chr(key), key))
    capture.release()
    cv2.destroyAllWindows()

canny_threshold_1 = 51
canny_threshold_2 = 301
delta = 10

filename = utils.ui_select_video()
ejemplo(filename)

print("FIN")

Ejemplo 4 - Difference of Gaussians
==

In [ ]:
def ejemplo(filename):
    global sigma1, sigma2, threshold
    capture = utils.abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        utils.mostrar_imagen("VIDEO", frame_gris)
        #calcular DoG
        blur1 = cv2.GaussianBlur(frame_gris, (sigma1, sigma1), 0)
        blur2 = cv2.GaussianBlur(frame_gris, (sigma2, sigma2), 0)
        frame_diff = cv2.subtract(blur1, blur2)
        utils.mostrar_imagen("Diff", utils.escalar(frame_diff, escalarMin0Max255=True))
        threshold = round(threshold, 2)
        th, frame_bin = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        utils.agregar_texto(frame_bin, "th={}".format(threshold))
        utils.mostrar_imagen("BIN", utils.escalar(frame_bin, escalarMin0Max255=True))
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            threshold += delta
        elif key == ord('z'):
            if threshold - delta > 0:
                threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sigma1 = 3
sigma2 = 13
threshold = 5
delta = 0.05

filename = utils.ui_select_video()
ejemplo(filename)

print("FIN")